# 1) DATENSAMMLUNG

In [55]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
headers = ['brand','model', 'price', 'year', 'transmission', 'fuelConsumption', 'tax', 'mileage', 'fuelType', 'color', 'variant']
df = pd.DataFrame(columns = headers)


In [56]:
colors = ['beige', 'black', 'blue', 'bronze', 'brown', 'gold', 'green', 'grey', 'orange', 'purple', 'red', 'silver', 'turquoise', 'white', 'yellow']
cars = ['toyota', 'volvo', 'audi', 'bmw', 'citroen', 'ford', 'honda', 'hyundai', 'kia', 'nissan']
url = "https://www.exchangeandmart.co.uk/used-cars-for-sale/"

In [ ]:
for color in colors:
    for pageNumber in range(1,100):
        for car in cars:
            currentUrl = url + car + "/any-distance-from-se15-1la/colour-" + color + '/page' + str(pageNumber)
            page = requests.get(currentUrl)
            soup = BeautifulSoup(page.text, 'html')
            items = soup.find_all('div', attrs={"class" : "result-item"})

            if items[0].find('span', attrs={"class" : "ttl_mk"}) is None:
                break
            
            for item in items:
                brand = item.find('span', attrs={"class" : "ttl_mk"}).text
                model = item.find('span', attrs={"class" : "ttl_md"}).text
                price = item.find('span', attrs={"class" : "price--primary"}).text
                variant = item.find('div', attrs={"class" : "result-item__variant"}).text
                keyDetails = item.find_all('p', attrs={"class" : "key-details__item"})
                if(len(keyDetails) < 6):
                    continue
                year = keyDetails[0].text
                transmission = keyDetails[1].text
                fuelConsumption = keyDetails[2].text
                tax = keyDetails[3].text
                mileage = keyDetails[4].text
                fueltype = keyDetails[5].text
                data = [brand, model, price, year, transmission, fuelConsumption, tax, mileage, fueltype, color, variant]
                df.loc[len(df)-1] = data

In [ ]:
df.to_csv('AllData.csv')
groupedDFs = [v for k, v in df.groupby('brand')]
for brandDF in groupedDFs:
    brandDF.to_csv(''+brandDF.brand.iloc[0]+'.csv') 

# 2) DATENBEREINIGUNG

**1.Was beschreiben die Daten?**

Wir können die auf der Website verfügbaren Daten über Autos scrapen. um einen Datensatz zu erstellen. Zu diesen Features der Daten gehören Marke, Modell, Preis, Jahr, Getriebe, Kraftstoffverbrauch, Steuer, Kilometerstand, Kraftstoffart, Farbe und Variante.

**2.Warum sind die Daten relevant oder interessant?**

Die Relevanz oder das Interesse der Daten hängt vom Kontext der Nutzung ab. Hier sind vier mögliche Gründe, warum der zur Verfügung gestellte Datensatz über Autos als relevant oder interessant angesehen werden kann:

* **Marktanalyse:** Die Daten könnten zur Analyse des Automarktes verwendet werden, um Preistrends, beliebte Modelle und den Einfluss von Faktoren wie Kraftstoffart oder Getriebe auf die Preisgestaltung zu verstehen.
* **Entscheidungsfindung:** Personen, die Autos kaufen oder verkaufen möchten, können den Datensatz nutzen, um Entscheidungen auf der Grundlage historischer Preistrends, des Kilometerstandes und anderer Features zu treffen.
* **Politik und Besteuerung:** Für Regierungen können die Daten nützlich sein, um die Steuerpolitik in Bezug auf Autos zu bewerten, insbesondere im Hinblick auf Faktoren wie Kraftstoffverbrauch oder Emissionen.
* **Einblicke in die Automobilbranche:** Unternehmen der Automobilindustrie können die Daten nutzen, um Einblicke in die Präferenzen der Verbraucher, die Beliebtheit der Modelle und die Preisstrategien der Wettbewerber zu gewinnen.

**3.Welche Daten sind enthalten?**

* Die Features über Autos sind Marke, Modell, Preis, Jahr, Getriebe, Kraftstoffverbrauch, Steuern, Kilometerstand, Kraftstofftyp, Farbe und Variante. 

* Einige von Features sind **kategorisch (oder qualitativ)**. Zum Beispiel: Farbe, Jahr, Marke 

* Einige von Features  sind **numerisch (oder quantitativ)** . Zum Beispiel: Preis, Steuer, Kraftstoffverbrauch

**4.Wie sind die Daten formatiert?**

Die gescrapten Daten enthalten Zeichenketten (String) und numerische (float64) Formate. Nur das Jahr wird als float64 gespeichert, die anderen Daten sind Strings. Daher sollten sie vor der Verwendung richtig vorbereitet und formatiert werden.

**5.Welche Schritte haben Sie unternommen, um die Daten zu bereinigen?**

Wir haben diese Operationen durchgeführt, um die Daten zu bereinigen: 
* Datenbereinigung, wie z. B. Behandlung fehlender Werte 
* Behandlung von Ausreißern
* Konvertierung von Datentypen
* Behandlung von Inkonsistenzen

**6.Gibt es Ausreißer, Duplikate oder fehlende Werte?**

* Ausreißer sind Werte, die deutlich von der Mehrheit abweichen. Wir identifizierten und behandelten die Ausreißer in unserem Datensatz mithilfe des z-Scores.

* Um doppelte Zeilen zu entfernen und sicherzustellen, dass es keine doppelten Zeilen gibt, haben wir die Methode **Pandas dataframe.drop_duplicates()** verwendet.

* Um mit fehlenden Werten umzugehen, haben wir die Werte durch Imputation richtig aufgefüllt oder sie direkt gelöscht. 

**7.Gibt es Fehler im Datensatz?**

Bei der Formatierung und Konvertierung der Daten mit der Methode replace() für Strings haben wir auch auf Fehler und Inkonsistenzen geprüft. Dank der ordnungsgemäß konfigurierten Website sind wir nicht auf falsche Daten getroffen.

**8.Sind die Daten konsistent?**

Um die Konsistenz des Datensatzes zu bestimmen, haben wir verschiedene Aspekte des Datensatzes bewertet. Hier sind die Schritte, die wir zur Bewertung der Datenkonsistenz durchgeführt haben:

***Einheitlichkeit der Datentypen:***

Wir haben überprüft, ob die Datentypen innerhalb jeder Spalte konsistent sind. So haben wir zum Beispiel sichergestellt, dass alle Werte in der Spalte "year" numerisch sind und gültige Jahre darstellen.

***Kategorische Werte:***

Wir haben kategorische Variablen wie "transmission", "fuelType", and "color".  untersucht. Es wurde sichergestellt, dass die Kategorien konsistent sind und keine Abweichungen aufgrund von Rechtschreibfehlern oder Synonymen vorliegen.

***Numerische Werte:***

Bei quantitativen Merkmalen wie "price", "mileage" und "fuelConsumption" haben wir den Wertebereich untersucht. Wir haben auf Ausreißer geprüft, die auf Fehler oder Unstimmigkeiten hindeuten können.

***Cross-Column Beziehungen:***

Wir haben die Beziehungen zwischen den Spalten untersucht. So haben wir beispielsweise überprüft, ob das "year" eines Autos mit dem "mileage" konsistent ist, und ob das "transmission" mit dem "fuelType" zusammenpasst.

***Fehlende Werte:***

Wir haben geprüft, ob es konsistente Muster bei fehlenden Werten gibt. Fehlende Werte können auf Probleme bei der Datenerfassung oder Dateneingabe hinweisen.

***Logische Konsistenz:***

Wir haben die logische Konsistenz der Daten bewertet. Beispielsweise wurde es überprüft, ob die Steuerwerte mit dem Jahr des Fahrzeugs übereinstimmen.

**9.Welche Besonderheiten sind Ihnen aufgefallen?**
*TODO*
* Die Steuer für die meisten Firmenwagen beträgt 0£.

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
import statsmodels.api as sm

In [4]:
df= pd.read_csv('AllData.csv')
df = df.drop('Unnamed: 0', axis=1)
df.rename(columns = {'fueltype':'fuelType'}, inplace = True) 
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37147 entries, 0 to 37146
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   brand            37147 non-null  object 
 1   model            37147 non-null  object 
 2   price            37147 non-null  object 
 3   year             37056 non-null  float64
 4   transmission     37147 non-null  object 
 5   fuelConsumption  37147 non-null  object 
 6   tax              37147 non-null  object 
 7   mileage          37147 non-null  object 
 8   fuelType         37147 non-null  object 
 9   color            37147 non-null  object 
 10  variant          36549 non-null  object 
dtypes: float64(1), object(10)
memory usage: 3.1+ MB


(37147, 11)

In [5]:
df.head()

,brand,model,price,year,transmission,fuelConsumption,tax,mileage,fuelType,color,variant
0,Toyota,Aygo,"£20,430",2022.0,Automatic,56.5 mpg,Tax: £180,"Mileage: 4,497",Petrol,beige,o X 1.0 VVT-i Exclusive 5dr Auto
1,Toyota,Aygo,"£17,998",2023.0,Automatic,56.5 mpg,Tax: £180,"Mileage: 2,345",Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto
2,Toyota,Aygo,"£14,225",2022.0,Manual,58.9 mpg,Tax: £180,"Mileage: 36,729",Petrol,beige,o X 1.0 VVT-i Edge 5dr
3,Toyota,Aygo,"£18,431",2023.0,Automatic,56.5 mpg,Tax: £180,"Mileage: 3,620",Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto
4,Toyota,Aygo,"£17,159",2022.0,Manual,57.7 mpg,Tax: £180,"Mileage: 4,000",Petrol,beige,o X 1.0 VVT-i Exclusive 5dr


In [6]:
#Formatting the price data / Bereinigung der Preisdaten
df['price'] = df['price'].str.replace(',','')
df['price'] = df['price'].str.replace('£','')
df['price'] = pd.to_numeric(df['price'])
#Formatting the year data 
df["year"].dropna(inplace=True)
df['year'] = df['year'].astype('int', errors='ignore')
#Formatting the fuelConsumption data 
df['fuelConsumption'] = df['fuelConsumption'].str.replace(' mpg','')
df['fuelConsumption'] = pd.to_numeric(df['fuelConsumption'])
#Formatting the Tax data 
df['tax'] = df['tax'].str.replace('Tax: n/a','')
df['tax'] = df['tax'].str.replace('Tax: £','')
df['tax'] = pd.to_numeric(df['tax'])
#Formatting the mileage data 
df['mileage'] = df['mileage'].str.replace(',','')
df['mileage'] = df['mileage'].str.replace('Mileage: ','')
#df['mileage'] = df['mileage'].str.replace('Mileage: Unknown','')
df['mileage'] = df['mileage'].str.replace('Unknown','')
df['mileage'] = pd.to_numeric(df['mileage'])

In [7]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37147 entries, 0 to 37146
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   brand            37147 non-null  object 
 1   model            37147 non-null  object 
 2   price            37147 non-null  int64  
 3   year             37056 non-null  float64
 4   transmission     37147 non-null  object 
 5   fuelConsumption  37147 non-null  float64
 6   tax              36646 non-null  float64
 7   mileage          37136 non-null  float64
 8   fuelType         37147 non-null  object 
 9   color            37147 non-null  object 
 10  variant          36549 non-null  object 
dtypes: float64(4), int64(1), object(6)
memory usage: 3.1+ MB


(37147, 11)

In [8]:
df.head()

,brand,model,price,year,transmission,fuelConsumption,tax,mileage,fuelType,color,variant
0,Toyota,Aygo,20430,2022.0,Automatic,56.5,180.0,4497.0,Petrol,beige,o X 1.0 VVT-i Exclusive 5dr Auto
1,Toyota,Aygo,17998,2023.0,Automatic,56.5,180.0,2345.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto
2,Toyota,Aygo,14225,2022.0,Manual,58.9,180.0,36729.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr
3,Toyota,Aygo,18431,2023.0,Automatic,56.5,180.0,3620.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto
4,Toyota,Aygo,17159,2022.0,Manual,57.7,180.0,4000.0,Petrol,beige,o X 1.0 VVT-i Exclusive 5dr


In [9]:
pd.isnull(df).sum()

brand                0
model                0
price                0
year                91
transmission         0
fuelConsumption      0
tax                501
mileage             11
fuelType             0
color                0
variant            598
dtype: int64

In [10]:
# Behandlung von fehlenden Werte
df["tax"].fillna(0, inplace=True)
df["mileage"].fillna(df["mileage"].mean(), inplace=True)
df["year"].fillna(df["year"].mean(), inplace=True)
df["variant"].fillna("unknown variant", inplace=True)

In [11]:
pd.isnull(df).sum()

brand              0
model              0
price              0
year               0
transmission       0
fuelConsumption    0
tax                0
mileage            0
fuelType           0
color              0
variant            0
dtype: int64

In [12]:
# Behandlung der Duplikate
display(df.drop_duplicates())
df.info()
df.shape

,brand,model,price,year,transmission,fuelConsumption,tax,mileage,fuelType,color,variant
0,Toyota,Aygo,20430,2022.0,Automatic,56.5,180.0,4497.0,Petrol,beige,o X 1.0 VVT-i Exclusive 5dr Auto
1,Toyota,Aygo,17998,2023.0,Automatic,56.5,180.0,2345.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto
2,Toyota,Aygo,14225,2022.0,Manual,58.9,180.0,36729.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr
3,Toyota,Aygo,18431,2023.0,Automatic,56.5,180.0,3620.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto
4,Toyota,Aygo,17159,2022.0,Manual,57.7,180.0,4000.0,Petrol,beige,o X 1.0 VVT-i Exclusive 5dr
...,...,...,...,...,...,...,...,...,...,...,...
37142,Toyota,Yaris,28650,2022.0,Automatic,65.7,170.0,2013.0,Hybrid,yellow,s Cross 1.5 Hybrid Dynamic AWD 5dr CVT
37143,Toyota,Aygo,7116,2016.0,Manual,69.0,0.0,50611.0,Petrol,yellow,1.0 VVT-i x-cite 3 Yellow Bi-Tone Hatchback 5d...
37144,Toyota,Yaris,24498,2022.0,Automatic,65.7,170.0,18002.0,Petrol,yellow,s 1.5 VVT h Dynamic E CVT Euro 6 (s/s) 5dr
37145,Toyota,Aygo,7500,2016.0,Manual,69.0,0.0,45336.0,Petrol,yellow,o 1.0 VVT-i X-Cite 3 5dr


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37147 entries, 0 to 37146
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   brand            37147 non-null  object 
 1   model            37147 non-null  object 
 2   price            37147 non-null  int64  
 3   year             37147 non-null  float64
 4   transmission     37147 non-null  object 
 5   fuelConsumption  37147 non-null  float64
 6   tax              37147 non-null  float64
 7   mileage          37147 non-null  float64
 8   fuelType         37147 non-null  object 
 9   color            37147 non-null  object 
 10  variant          37147 non-null  object 
dtypes: float64(4), int64(1), object(6)
memory usage: 3.1+ MB


(37147, 11)

In [14]:
# Behandlung der Ausreißer
# Kalkulieren von 'Z-score' für jeden Feature
df["z_score_price"]=(df["price"] - df["price"].mean())/df["price"].std()
df["z_score_year"]=(df["year"] - df["year"].mean())/df["year"].std()
df["z_score_fuelConsumption"]=(df["fuelConsumption"] - df["fuelConsumption"].mean())/df["fuelConsumption"].std()
df["z_score_tax"]=(df["tax"] - df["tax"].mean())/df["tax"].std()
df["z_score_mileage"]=(df["mileage"] - df["mileage"].mean())/df["mileage"].std()
df.head()

,brand,model,price,year,transmission,fuelConsumption,tax,mileage,fuelType,color,variant,z_score_price,z_score_year,z_score_fuelConsumption,z_score_tax,z_score_mileage
0,Toyota,Aygo,20430,2022.0,Automatic,56.5,180.0,4497.0,Petrol,beige,o X 1.0 VVT-i Exclusive 5dr Auto,0.105113,0.848492,0.034788,0.19661,-1.002342
1,Toyota,Aygo,17998,2023.0,Automatic,56.5,180.0,2345.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto,-0.119828,1.132588,0.034788,0.19661,-1.078382
2,Toyota,Aygo,14225,2022.0,Manual,58.9,180.0,36729.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr,-0.468799,0.848492,0.122478,0.19661,0.136568
3,Toyota,Aygo,18431,2023.0,Automatic,56.5,180.0,3620.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto,-0.079779,1.132588,0.034788,0.19661,-1.033330
4,Toyota,Aygo,17159,2022.0,Manual,57.7,180.0,4000.0,Petrol,beige,o X 1.0 VVT-i Exclusive 5dr,-0.197428,0.848492,0.078633,0.19661,-1.019903


In [15]:
# Löschen der Ausreißer
df_filtered = df[
    (df["z_score_price"] > -3) & (df["z_score_price"] < 3) &
    (df["z_score_year"] > -3) & (df["z_score_year"] < 3) &
    (df["z_score_fuelConsumption"] > -3) & (df["z_score_fuelConsumption"] < 3) &
    (df["z_score_tax"] > -3) & (df["z_score_tax"] < 3) &
    (df["z_score_mileage"] > -3) & (df["z_score_mileage"] < 3)
]

df_filtered.drop(["z_score_price", "z_score_year", "z_score_fuelConsumption", "z_score_tax", "z_score_mileage"], axis=1, inplace=True)

df = df_filtered

C:\Users\aykan\AppData\Local\Temp\ipykernel_34420\3429897757.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(["z_score_price", "z_score_year", "z_score_fuelConsumption", "z_score_tax", "z_score_mileage"], axis=1, inplace=True)


In [16]:
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Index: 34422 entries, 0 to 37146
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   brand            34422 non-null  object 
 1   model            34422 non-null  object 
 2   price            34422 non-null  int64  
 3   year             34422 non-null  float64
 4   transmission     34422 non-null  object 
 5   fuelConsumption  34422 non-null  float64
 6   tax              34422 non-null  float64
 7   mileage          34422 non-null  float64
 8   fuelType         34422 non-null  object 
 9   color            34422 non-null  object 
 10  variant          34422 non-null  object 
dtypes: float64(4), int64(1), object(6)
memory usage: 3.2+ MB


(34422, 11)

In [17]:
df.head()

,brand,model,price,year,transmission,fuelConsumption,tax,mileage,fuelType,color,variant
0,Toyota,Aygo,20430,2022.0,Automatic,56.5,180.0,4497.0,Petrol,beige,o X 1.0 VVT-i Exclusive 5dr Auto
1,Toyota,Aygo,17998,2023.0,Automatic,56.5,180.0,2345.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto
2,Toyota,Aygo,14225,2022.0,Manual,58.9,180.0,36729.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr
3,Toyota,Aygo,18431,2023.0,Automatic,56.5,180.0,3620.0,Petrol,beige,o X 1.0 VVT-i Edge 5dr Auto
4,Toyota,Aygo,17159,2022.0,Manual,57.7,180.0,4000.0,Petrol,beige,o X 1.0 VVT-i Exclusive 5dr
